<a href="https://colab.research.google.com/github/FrederickUdis/SegmentacionSemantica/blob/main/modeloDeeplab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalaciones Necesarias

In [1]:
!pip install nnabla opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

## Librerias Usadas

In [2]:
import cv2
import os
import numpy as np
import nnabla as nn
import cv2
from nnabla.models.semantic_segmentation import DeepLabV3plus
from nnabla.models.semantic_segmentation.utils import ProcessImage
import os

## Funcion para convertir Video en fotogramas

In [4]:
def dividir_video_en_fotogramas(ruta_video, ruta_salida, fps=60):
    # Cargar el video
    video = cv2.VideoCapture(ruta_video)

    # Verificar si el video se abrió correctamente
    if not video.isOpened():
        print("Error al abrir el archivo de video.")
        return

    # Obtener FPS del video original
    video_fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames del video",video_fps)
    # Calcular el intervalo de frames a capturar
    frame_interval = int(video_fps / fps)

    # Crear el directorio de salida si no existe
    os.makedirs(ruta_salida, exist_ok=True)

    frame_id = 0
    count = 0
    while True:
        # Leer un fotograma
        success, frame = video.read()

        # Verificar si la lectura fue exitosa
        if not success or count > video_fps:
            break

        if count % frame_interval == 0:
            # Guardar el fotograma como imagen
            cv2.imwrite(f"{ruta_salida}/frame_{frame_id}.jpg", frame)
            frame_id += 1

        count += 1

    # Liberar el objeto VideoCapture
    video.release()
    print(f"Fotogramas extraídos en {ruta_salida}")



In [8]:
# Uso de la función
target_h, target_w = 513, 513
dividir_video_en_fotogramas('/content/video_entrada.mp4', '/content/frames/')

Frames del video 60.0
Fotogramas extraídos en /content/frames/


## Modelo preEntrenado para segmentacion semantica

In [9]:
deeplabv3 = DeepLabV3plus('voc-coco', output_stride=8)
x = nn.Variable((1, 3, target_h, target_w), need_grad=False)
y = deeplabv3(x)

Loading /root/nnabla_data/nnp_models/semantic_segmentation/DeepLabV3-voc-coco-os-8.nnp.


## Aplicacion del modelo a cada fotograma

In [10]:
N=60
for i in range(N):
  filename = f"/content/frames/frame_{i}.jpg"
  if not os.path.exists(filename):
      continue

  # Cargar la imagen usando OpenCV
  image = cv2.imread(filename)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB

  # Preprocesamiento de la imagen
  processed_image = ProcessImage(image, target_h, target_w)
  input_array = processed_image.pre_process()

  # Inferencia
  x.d = input_array
  y.forward(clear_buffer=True)
  output = np.argmax(y.d, axis=1)

  # Postprocesamiento
  post_processed = processed_image.post_process(output[0])

  # Mostrar clases segmentadas
  predicted_classes = np.unique(post_processed).astype(int)
  print(f"Clase Segmentada en {filename}:")
  for cls in predicted_classes:
      print(deeplabv3.category_names[cls])

  # Guardar el resultado de la segmentación
  output_filename = f"/content/output_frame_{i}.png"
  processed_image.save_segmentation_image(output_filename)

Clase Segmentada en /content/frames/frame_0.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_1.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_2.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_3.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_4.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_5.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_6.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_7.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_8.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_9.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_10.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_11.jpg:
aeroplane
cow
dog
pottedplant
Clase Segmentada en /content/frames/frame_12.jpg:


## Creacion del video apartir de los nuevos fotogramas

In [15]:
def crear_video_desde_fotogramas(directorio_fotogramas, salida_video, fps=24):
    # Verificar si el directorio existe
    if not os.path.exists(directorio_fotogramas):
        print(f"El directorio {directorio_fotogramas} no existe.")
        return

    # Obtener lista de fotogramas
    fotogramas = [img for img in os.listdir(directorio_fotogramas) if img.endswith(".png")]

    # Verificar si hay fotogramas en el directorio
    if not fotogramas:
        print("No se encontraron fotogramas en el directorio.")
        return

    # Asegurarse de que los fotogramas se ordenen numéricamente
    fotogramas.sort(key=lambda x: int(x.split('_')[2].split('.')[0]))

    # Leer el primer fotograma para determinar el tamaño
    frame = cv2.imread(os.path.join(directorio_fotogramas, fotogramas[0]))
    if frame is None:
        print("Error al leer el primer fotograma.")
        return
    altura, ancho, capas = frame.shape
    tamaño = (ancho, altura)

    # Crear el objeto VideoWriter
    video = cv2.VideoWriter(salida_video, cv2.VideoWriter_fourcc(*'mp4v'), fps, tamaño)

    # Añadir fotogramas al video
    for fotograma in fotogramas:
        imagen = cv2.imread(os.path.join(directorio_fotogramas, fotograma))
        if imagen is not None:
            video.write(imagen)

    # Liberar el objeto VideoWriter
    video.release()
    print(f"Fotogramas extraídos en {salida_video}")


In [17]:
crear_video_desde_fotogramas('/content/', '/content/video_salida.mp4')

Fotogramas extraídos en /content/video_salida.mp4
